In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, Output, interact_manual
import textwrap
from screeninfo import get_monitors

In [46]:
# Global variables
principle_widgets = []
principles_for = []
principles_against = []
all_widgets = []  # To keep track of all created widgets
default_values = {
    'length': 50,
    'thickness': 1.0
}
legal_status_for = ""
legal_status_against = ""
act = ""

# Function to create widgets for each principle
def create_principle_widgets():
    global principle_widgets
    principle_widgets = []

    for label in principles_for:
        length_widget = widgets.IntSlider(
            min=1, max=100, step=1, value=default_values['length'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        thickness_widget = widgets.FloatSlider(
            min=0.1, max=5, step=.1, value=default_values['thickness'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        principle_widgets.append((length_widget, thickness_widget, label, 1))

    for label in principles_against:
        length_widget = widgets.IntSlider(
            min=1, max=100, step=1, value=default_values['length'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        thickness_widget = widgets.FloatSlider(
            min=0.1, max=5, step=0.1, value=default_values['thickness'],
            description=label,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px')
        )
        principle_widgets.append((length_widget, thickness_widget, label, -1))

# Function to plot principles based on current slider values
def plot_principles(save_path=None):
    with output:
        output.clear_output(wait=True)
        fig, ax = plt.subplots()

        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

        num_is = len(principles_for)
        num_not = len(principles_against)
        max_num = max(num_is, num_not)

        if max_num >= 2:
            lim = max_num + 0.5
        else:
            lim = 1.5

        ax.set_xlim(0, lim)
        ax.set_ylim(0, lim)

        y_offset_is = lim/(num_is+1)
        y_offset_not = lim/(num_not+1)
        sum_for = 0
        sum_against = 0

        for length_widget, thickness_widget, label, direction in principle_widgets:
            length = lim/2 * 0.9 * (length_widget.value/100)
            thickness = 5*(thickness_widget.value)
            sum_length = length_widget.value
            sum_thickness = thickness_widget.value
            x_start = 0 +(lim *.05) if direction == 1 else lim-(lim*0.05)
            x_end = x_start + direction * length

            arrow_style = f"Simple,tail_width={0.5},head_width={1.1},head_length={1.25}"

            if direction == 1:
                arrow = patches.FancyArrowPatch((x_start, y_offset_is), (x_end, y_offset_is),
                                                connectionstyle="arc3",
                                                arrowstyle=arrow_style,
                                                color='lightblue',
                                                mutation_scale=thickness,
                                                linewidth=thickness)
                ax.add_patch(arrow)
                ax.text((x_start + x_end) / 2, y_offset_is-0.02, label, fontsize=10, ha='center')
                y_offset_is += lim/(num_is+1)
                sum_for += (sum_length/100)*(sum_thickness)

            else:
                arrow = patches.FancyArrowPatch((x_start, y_offset_not), (x_end, y_offset_not),
                                                connectionstyle="arc3",
                                                arrowstyle=arrow_style,
                                                color='orange',
                                                mutation_scale=thickness,
                                                linewidth=thickness)
                ax.add_patch(arrow)
                ax.text((x_start + x_end) / 2, y_offset_not-0.02, label, fontsize=10, ha='center')
                y_offset_not += lim/(num_not+1)
                sum_against += (sum_length/100)*(sum_thickness)

        ax.text(lim/2- lim*0.25, lim + lim*0.05, legal_status_for, fontsize=18, ha='center', va='center', color='orange')
        ax.text(lim/2+lim*0.25, lim + lim*0.05, legal_status_against, fontsize=18, ha='center', va='center', color='lightblue')
        ax.axvline(x=lim/2, color='gray', linestyle='--')

        circle_position = (sum_for - sum_against)*(lim/50) + lim/2
        radius = 0.1*lim
        circle = patches.Circle((circle_position, lim/2), radius=radius, color='green', alpha=0.5)
        ax.add_patch(circle)

        ax.text(circle_position, lim / 2, act, fontsize=10, ha='center', va='center', color='black')

        ax.grid(False)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_aspect('equal')

        plt.tight_layout()

        if save_path:
            fig.savefig(save_path, bbox_inches='tight')

        plt.show()

# Function to clear the plot and remove all widgets
def clear_plot():
    global principles_for, principles_against, legal_status_for, legal_status_against, act, principle_widgets, all_widgets

    # Clear the output area
    output.clear_output(wait=True)

    # Close and remove all widgets from display
    for widget in all_widgets:
        widget.close()
    
    # Clear all widget lists
    all_widgets = []
    principle_widgets = []

    # Reset all data
    principles_for = []
    principles_against = []
    legal_status_for = ""
    legal_status_against = ""
    act = ""

    # Recreate the principle widgets
    create_principle_widgets()

    # (Re-)display the widgets and plot, if necessary
    display_widgets()
    plot_principles()

# Creating text input widgets
style = {'description_width': 'initial'}

#Text boxes for headings, principles, and act
text_input_status_for = widgets.Text(
    value='',
    placeholder=legal_status_for,
    description="Set legal status:",
    disabled=False,
    style=style
)
text_input_status_against = widgets.Text(
    value='',
    placeholder=legal_status_against,
    description="Set legal status:",
    disabled=False,
    style=style
)

text_input_for = widgets.Text(
    value='',
    placeholder='Enter principle',
    description=f'Principle supporting {legal_status_against}:',
    disabled=False,
    style=style
)

text_input_against = widgets.Text(
    value='',
    placeholder='Enter principle',
    description=f'Principle supporting {legal_status_for}:',
    disabled=False,
    style=style
)

set_act = widgets.Text(
    value='',
    placeholder='',
    description="Set Act",
    disabled=False,
    style=style
)

#allowing you to save the plot
text_save_plot = widgets.Text(
    value='',
    placeholder='Enter file name',
    description='Save Plot (default is on desktop)',
    disabled=False,
    style=style
)

#Function triggered when changing act
def update_act(new_act):
    global act

    if new_act:
        act = new_act
        plot_principles()
        set_act.value = ''

#Function triggered to change headings
def update_statuses(status_for, status_against):
    global legal_status_for, legal_status_against

    if status_for:
        legal_status_for = status_for
    if status_against:
        legal_status_against = status_against
    if status_for and status_against:
        plot_principles()
        text_input_status_for.value = ''
        text_input_status_against.value = ''

# Function to update principles and plot
def update_plot(principle_for, principle_against):
    global principles_for, principles_against
    if principle_for:
        principles_for.append(principle_for)
        text_input_for.value = ''

    if principle_against:
        principles_against.append(principle_against)
        text_input_against.value = ''

    if principle_for or principle_against:
        plot_principles()
        create_principle_widgets()
        display_widgets()

# Function to save the plot
def save_plot(file_path):
    if file_path:
        try:
            plot_principles(file_path)
            text_save_plot.value = ''
            text_save_plot.placeholder = "Saved!"
        except:
            text_save_plot.value = ''
            text_save_plot.placeholder = f"{file_path} is not a valid path"


# Using interact_manual for better control with a custom button label. For act
ym = interact_manual.options(manual_name = "Set act")
ym(update_act, new_act=set_act)

#For headings
xm = interact_manual.options(manual_name = "Set contrasting legal statuses")
xm(update_statuses, status_for=text_input_status_for, status_against=text_input_status_against)

#For principles
im = interact_manual.options(manual_name = "Add principles")
im(update_plot, principle_for=text_input_for, principle_against=text_input_against)

# Creating the output widget to display the plot
output = widgets.Output()
display(output)

# Clear button
clear_button = widgets.Button(description='Clear Plot')
clear_button.on_click(lambda x: clear_plot())
display(clear_button)

#For save
zm = interact_manual.options(manual_name ="Save plot")
zm(save_plot, file_path = text_save_plot)

# Function to display all the widgets
def display_widgets():
    global all_widgets

    # Clear any previously stored widgets
    all_widgets = []

    length_widgets = widgets.VBox([
        widgets.Label(value='Percentage of Activation', layout=widgets.Layout(font_weight='bold')),
        *[widgets.HBox([length_widget]) for length_widget, _, _, _ in principle_widgets]
    ])

    thickness_widgets = widgets.VBox([
        widgets.Label(value='Relative Importance', layout=widgets.Layout(font_weight='bold')),
        *[widgets.HBox([thickness_widget]) for _, thickness_widget, _, _ in principle_widgets]
    ])

    widgets_box = widgets.HBox([length_widgets, thickness_widgets])

    # Track the widgets
    all_widgets.append(length_widgets)
    all_widgets.append(thickness_widgets)
    all_widgets.append(widgets_box)

    # Display widgets
    display(widgets_box)

    # Re-attaching observers to all sliders
    for length_widget, thickness_widget, _, _ in principle_widgets:
        length_widget.observe(on_value_change, names='value')
        thickness_widget.observe(on_value_change, names='value')

# Function to handle value changes on sliders
def on_value_change(change):
    plot_principles()

# Initialize by creating widgets for principles and displaying them
create_principle_widgets()
#display_widgets()
plot_principles()

interactive(children=(Text(value='', continuous_update=False, description='Set Act', placeholder='', style=Tex…

interactive(children=(Text(value='', continuous_update=False, description='Set legal status:', placeholder='',…

interactive(children=(Text(value='', continuous_update=False, description='Principle supporting :', placeholde…

Output()

Button(description='Clear Plot', style=ButtonStyle())

interactive(children=(Text(value='', continuous_update=False, description='Save Plot (default is on desktop)',…